# QuantumFold-Advantage: Complete Benchmarking Suite

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/complete_benchmark.ipynb)

**Version:** 1.0.0

**This notebook runs EVERYTHING:**
- Full training pipeline (quantum vs classical)
- Comprehensive evaluation metrics
- Statistical validation with hypothesis testing
- Publication-ready visualizations
- Checkpointing and resume support

**Estimated runtime:** 30-60 minutes with GPU

**Author:** Tommaso R. Marena (The Catholic University of America)

## 1. Setup Environment

In [ ]:
# Check environment
try:
    import google.colab
    IN_COLAB = True
    print('✅ Running in Google Colab')
except ImportError:
    IN_COLAB = False
    print('💻 Running locally')

# Check GPU
import torch
print(f'\n🔥 PyTorch: {torch.__version__}')
print(f'⚡ CUDA: {torch.cuda.is_available()}')

if torch.cuda.is_available():
    print(f'🎮 GPU: {torch.cuda.get_device_name(0)}')
    print(f'💾 Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('⚠️  No GPU - training will be VERY slow')
    print('   Enable GPU: Runtime > Change runtime type > T4 GPU')
    
    response = input('Continue without GPU? (y/n): ')
    if response.lower() != 'y':
        raise RuntimeError('GPU required for reasonable training time')